# Домашнее задание №1

В этом домашнем задании вы познакомитесь с платформой Kaggle, а также поучаствуете в небольшом сореврновании, которое поможет закрепить вам знания, полученные на наших лекциях.

Соревнование можно найти по ссылке: https://www.kaggle.com/c/mytracker-mlhomework1-fall20 \
Для того, чтобы присоединиться к этому соревнованию, перейдите по ссылке: https://www.kaggle.com/t/ebaddc30cce54e329c64f7504b274c2c \
Скачать данные можно во вкладке `Data`, в которой находится 3 файла:
 - `train.csv` - обучающая выборка
 - `test.csv` - тестовая выборка
 - `sampleSubmission.csv` - пример сабмита, который вы должны будете загрузить на Kaggle, чтобы предоставить свое решение (аналогичный файл мы с вами сгенерируем в этом ноутбуке)
 
Описание данных вы можете прочитать также на Kaggle.

Несколько основных моментов данного домашнего задания:
 1. Метрикой является RMSE (Root Mean Square Error, https://en.wikipedia.org/wiki/Root-mean-square_deviation).
 2. Тестовая выборка из файла `test.csv` разделена на 2 части: 
  - __Public Leaderboard__ - во время участия вы можете ориентироваться на этот лидерборд, чтобы оценивать ваше решение, а также видеть несколько наших бенчмарков и решения других участников.\
  **Важный момент**. Оценка на Public Leaderboard строится не на всех примерах из `test.csv`, а только на 60% (каких именно узнать нельзя).
  - __Private Leaderboard__ - откроется только после окончания соревнования. Именно по нему мы будем распределять баллы и он состоит из оставшихся 40% примеров из `test.csv`.
 3. Мы надеемся на то, что вы будете не только использовать модели машинного обучения, которые мы будем проходить по ходу лекций, но и попробуете сами изучить данные и сделать прогнозы "руками". Соревнование будет длиться достаточно долго (до 23.11), тем самым будет неплохо, если вы сможете изучить выданные датасеты, чтобы сделать более качественный прогноз.
 4. В день можно отправить до 5 посылок с решением, так что не оставляйте все на последний день.
 5. В конце соревнования ТОП10 участников мы попросим прислать рабочий ноутбук с их решением, чтобы убедиться, что никто не пытался сжульничать, так что держите свое решение "в чистоте", чтобы по окончанию соревнования его можно было показать нам.
 6. В соревновании нет ограничений на модели или подходы, которые вы будете использовать, если они не выходят за рамки данных, предоставленных в данном соревновании.

In [30]:
import pandas as pd
from sklearn.metrics import mean_squared_error
import math
from sklearn import ensemble
from sklearn.ensemble import RandomForestRegressor

In [31]:
dir_='/Users/admin/Desktop/ODS Course/mytracker_mlcourse/HW_%1/'
train = pd.read_csv(dir_+"train.csv")
test = pd.read_csv(dir_+"test.csv")
submission_filename = pd.read_csv(dir_+"sampleSubmission.csv")

In [32]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,Holiday
0,474,5,2019-04-13,4256,502,1,0,0.0
1,862,5,2019-04-13,2574,319,1,0,0.0
2,108,5,2019-04-13,4687,492,1,0,0.0
3,48,5,2019-04-13,9779,1370,1,0,0.0
4,539,5,2019-04-13,2156,234,1,0,0.0


In [73]:
test.head()

,Store,DayOfWeek,Date,Holiday
ID,,,,
1,12,6,2019-04-14,0
2,12,0,2019-04-15,0
3,12,1,2019-04-16,0
4,12,2,2019-04-17,0
5,12,3,2019-04-18,0


In [74]:
train.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,Holiday
count,963689.000000,963689.000000,963689.000000,963689.000000,963689.000000,963689.000000,832617.000000
mean,557.497102,3.000562,5754.265803,632.871642,0.828595,0.379562,0.037292
std,321.706529,1.997247,3851.690209,465.524255,0.376863,0.485278,0.189477
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,278.000000,1.000000,3703.000000,403.000000,1.000000,0.000000,0.000000
50%,558.000000,3.000000,5726.000000,609.000000,1.000000,0.000000,0.000000
75%,837.000000,5.000000,7838.000000,838.000000,1.000000,1.000000,0.000000
max,1115.000000,6.000000,38722.000000,7388.000000,1.000000,1.000000,1.000000


In [75]:
train[train['Open']==0].describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,Holiday
count,165181.000000,165181.000000,165181.0,165181.0,165181.0,165181.000000,146249.000000
mean,557.570259,5.304157,0.0,0.0,0.0,0.067532,0.206087
std,321.344155,1.575173,0.0,0.0,0.0,0.250942,0.404495
min,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
25%,279.000000,6.000000,0.0,0.0,0.0,0.000000,0.000000
50%,559.000000,6.000000,0.0,0.0,0.0,0.000000,0.000000
75%,836.000000,6.000000,0.0,0.0,0.0,0.000000,0.000000
max,1115.000000,6.000000,0.0,0.0,0.0,1.000000,1.000000


In [76]:
train[train['Customers']==0].describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,Holiday
count,165233.000000,165233.000000,165233.0,165233.0,165233.000000,165233.000000,146281.000000
mean,557.557552,5.303347,0.0,0.0,0.000315,0.067626,0.206042
std,321.346049,1.575756,0.0,0.0,0.017737,0.251103,0.404462
min,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000
25%,279.000000,6.000000,0.0,0.0,0.000000,0.000000,0.000000
50%,559.000000,6.000000,0.0,0.0,0.000000,0.000000,0.000000
75%,836.000000,6.000000,0.0,0.0,0.000000,0.000000,0.000000
max,1115.000000,6.000000,0.0,0.0,1.000000,1.000000,1.000000


In [77]:
train[train['Holiday']==0].describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,Holiday
count,801567.000000,801567.000000,801567.000000,801567.000000,801567.000000,801567.000000,801567.0
mean,557.555433,3.027975,5970.557016,654.313827,0.855147,0.380955,0.0
std,321.741140,2.006387,3794.637050,458.681089,0.351952,0.485622,0.0
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,279.000000,1.000000,3970.000000,429.000000,1.000000,0.000000,0.0
50%,558.000000,3.000000,5866.000000,623.000000,1.000000,0.000000,0.0
75%,837.000000,5.000000,7967.000000,850.000000,1.000000,1.000000,0.0
max,1115.000000,6.000000,38037.000000,7388.000000,1.000000,1.000000,0.0


Описание данных:
1. Store-номер магазина
2. DayOfWeek - день недели-можно применить OHE для более корректноо кодирования
3. Sales-  число продаж - наш таргет
4. Customers-число продавцов , если их нет - продаж соответственно тоже нет
5. Open -статус магазина ,  если он закрыт - продаж тоже нет
6. Promo-есть/нет промо акции
7. Holiday-праздник/отсутствие праздника


1) Сначала обучим случ лес , получим оценку , потом применим OHE, сравним оценку ,  потом попробуем обучиться на всех признаках, кроме open, так же удалим объекты, где число продавцов = 0. На предикте мы можем сами предсказать 0 в этих ситуациях.

In [78]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,Holiday
0,474,5,2019-04-13,4256,502,1,0,0.0
1,862,5,2019-04-13,2574,319,1,0,0.0
2,108,5,2019-04-13,4687,492,1,0,0.0
3,48,5,2019-04-13,9779,1370,1,0,0.0
4,539,5,2019-04-13,2156,234,1,0,0.0
...,...,...,...,...,...,...,...,...
963684,73,1,2016-11-01,0,0,0,0,1.0
963685,649,1,2016-11-01,0,0,0,0,1.0
963686,429,1,2016-11-01,0,0,0,0,1.0
963687,898,1,2016-11-01,0,0,0,0,1.0


In [268]:
date_hol =train[['Date','Holiday']].groupby(by='Date').median()
date_hol['Holiday']['2016-11-01']

### заменим наны:
def smart_fillna(df,numb):
    date_hol =df[['Date','Holiday']].groupby(by='Date').median()
    
    date_hol.Holiday=date_hol.Holiday.apply(lambda r:  numb  if math.isnan(r)== True else r)

    df.Holiday=df.apply(lambda r:  date_hol['Holiday'][r['Date']] 
                              if math.isnan(r['Holiday'])== True 
                              else r['Holiday'], axis=1)
    return df

train=smart_fillna(train,0.13)

In [100]:
print(date_hol.describe())
print(date_hol.Holiday.unique())
#date_hol.Holiday=date_hol.Holiday.fillna(0.5)


          Holiday
count  894.000000
mean     0.102349
std      0.237534
min      0.000000
25%      0.000000
50%      0.000000
75%      0.000000
max      1.000000
[1.  0.  0.5]


In [264]:
def just_fit_predict(X_train,y_train,test):
    regr = RandomForestRegressor(max_depth=3, random_state=1,n_estimators=30)
    regr.fit(X_train, y_train)
    prediction=regr.predict(test)
    test['ID']= [i for i in range(len(test))]
    submission = test['ID'].to_frame()
    submission['prediction'] = prediction
    submission.to_csv(dir_+'sampleSubmission.csv', index=None)

In [105]:
regr = RandomForestRegressor(max_depth=2, random_state=0,n_estimators=300)

In [109]:
data=train.copy()
test1=test.copy()
y=data['Sales']
del data['Sales']
X = data

del data['Date']
del test1['Date']
del X['Open']
del X['Promo']
del X['Customers']

print(X.columns)
print(test1.columns)

regr.fit(X, y)
prediction=regr.predict(test1)

Index(['Store', 'DayOfWeek', 'Holiday'], dtype='object')
Index(['Store', 'DayOfWeek', 'Holiday'], dtype='object')


In [115]:
submission.to_csv(dir_+'sampleSubmission.csv', index=None)

### MSE на сабмите - 2827.75707

# UPD №2 делаем EDA ,  смотрим на признаки и генерируем новые фичи

In [125]:
### Данные по дате не пересекаются ,
print(test.sort_values('Date').head())

print(train.sort_values('Date',ascending=False).head())

          ID  Store  DayOfWeek        Date  Holiday
0          1     12          6  2019-04-14        0
2496    2497    243          6  2019-04-14        0
10512  10513    931          6  2019-04-14        0
2448    2449    242          6  2019-04-14        0
10560  10561    934          6  2019-04-14        0
     Store  DayOfWeek        Date  Sales  Customers  Open  Promo  Holiday
0      474          5  2019-04-13   4256        502     1      0      0.0
742    674          5  2019-04-13   2181        249     1      0      0.0
748    782          5  2019-04-13   1866        261     1      0      0.0
747   1066          5  2019-04-13   5412        471     1      0      0.0
746    204          5  2019-04-13   6652        602     1      0      0.0


In [154]:
## В обучающей выборке есть все магазины из тестовой
list1=list(train.Store.unique())
print(len(list1))
list2=list(test.Store.unique())
print(len(list2))

result=list(set(list1) ^ set(list2))
len(result)

c = list(set(list1) & set(list2))
len(c)

1115
273


273

In [163]:
## Посмотрим на описание магазинов их тестовой выборки

stores_ = train[train['Store'].isin(list2)].reset_index(drop=True)
stores_

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,Holiday
0,862,5,2019-04-13,2574,319,1,0,0.0
1,492,5,2019-04-13,3456,397,1,0,0.0
2,456,5,2019-04-13,5565,582,1,0,0.0
3,238,5,2019-04-13,3615,392,1,0,0.0
4,266,5,2019-04-13,7996,689,1,0,0.0
...,...,...,...,...,...,...,...,...
235592,481,1,2016-11-01,0,0,0,0,1.0
235593,995,1,2016-11-01,0,0,0,0,1.0
235594,166,1,2016-11-01,0,0,0,0,1.0
235595,73,1,2016-11-01,0,0,0,0,1.0


In [205]:
print(len(stores_))
print(len(stores_[['Store','Date']].drop_duplicates()))

235597
235597


In [224]:
store_work_days=stores_.groupby(['Store','Open','DayOfWeek'])[['DayOfWeek']].count()
print(len(store_work_days))
store_work_days.head(50)

3382


DayOfWeek
Store Open DayOfWeek           
12    0    0                  7
           1                  2
           2                  3
           3                  9
           4                  7
           6                127
      1    0                120
           1                126
           2                125
           3                119
           4                121
           5                128
13    0    0                  7
           1                  2
           2                  3
           3                 10
           4                  5
           6                101
      1    0                 94
           1                 99
           2                 98
           3                 92
           4                 97
           5                102
14    0    0                  6
           1                  1
           2                  3
           3                  8
           4                  6
           6                127
      1    0                121
           1                127
           2                125
           3                120
           4                122
           5                128
29    0    0                  6
           1                  1
           2                  3
           3                 11
           4                  6
           6                127
      1    0                121
           1                127
           2                125
           3                117
           4                122
           5                128
40    0    0                  6
           1                  1

In [237]:

store_work_days.rename(columns={'DayOfWeek': 'count_DayOfWeek'}, inplace=True)
store_work_days.loc[(store_work_days['count_DayOfWeek'] > 90)&(store_work_days['count_DayOfWeek'] > 90)]


count_DayOfWeek
Store Open DayOfWeek                 
12    0    6                      127
      1    0                      120
           1                      126
           2                      125
           3                      119
...                               ...
1113  1    1                      127
           2                      125
           3                      120
           4                      122
           5                      128

[1901 rows x 1 columns]

In [247]:
#store_work_days=store_work_days.reset_index()
store_day_off=store_work_days[['Store','DayOfWeek','count_DayOfWeek']][(store_work_days['count_DayOfWeek']>90)&(store_work_days['Open']==0)]

store_day_off['DayOfWeek'].unique()
### Вот таким сложным путём мы узнали ,что магазины отдыхают только по вскр

array([6])

In [275]:
test['Open']=test.DayOfWeek.apply(lambda x: 0 if x==6 else 1)
test

,ID,Store,DayOfWeek,Date,Holiday,Open
0,1,12,6,2019-04-14,0,0
1,2,12,0,2019-04-15,0,1
2,3,12,1,2019-04-16,0,1
3,4,12,2,2019-04-17,0,1
4,5,12,3,2019-04-18,0,1
...,...,...,...,...,...,...
13099,13100,1113,0,2019-05-27,0,1
13100,13101,1113,1,2019-05-28,0,1
13101,13102,1113,2,2019-05-29,0,1
13102,13103,1113,3,2019-05-30,0,1


In [255]:
train[['Store','DayOfWeek','Open','Holiday']]

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,Holiday
0,474,5,2019-04-13,4256,502,1,0,0.0
1,862,5,2019-04-13,2574,319,1,0,0.0
2,108,5,2019-04-13,4687,492,1,0,0.0
3,48,5,2019-04-13,9779,1370,1,0,0.0
4,539,5,2019-04-13,2156,234,1,0,0.0
...,...,...,...,...,...,...,...,...
963684,73,1,2016-11-01,0,0,0,0,1.0
963685,649,1,2016-11-01,0,0,0,0,1.0
963686,429,1,2016-11-01,0,0,0,0,1.0
963687,898,1,2016-11-01,0,0,0,0,1.0


In [270]:
train=smart_fillna(train,1)

In [276]:
# just_fit_predict(X_train=train[['Store','DayOfWeek','Open','Holiday']],
#                  y_train=train['Sales'],
#                  test=test[['Store','DayOfWeek','Holiday','Open']])


regr = RandomForestRegressor(max_depth=2, random_state=1,n_estimators=300)
regr.fit(train[['Store','DayOfWeek','Open','Holiday']], train['Sales'])
prediction=regr.predict(test[['Store','DayOfWeek','Holiday','Open']])
submission = test['ID'].to_frame()
submission['prediction'] = prediction
submission.to_csv(dir_+'sampleSubmission.csv', index=None)

### Вывод, выходные зря добавил,  они явно мешают модели. На сабмите скор 7200

Можно ещё попробовать обучить модель , чтобы она предсказывала , открыт магазин или нет:

In [33]:
train = pd.read_csv(dir_+"train.csv")
test = pd.read_csv(dir_+"test.csv")


In [34]:
### Создадим уже признаки:
def get_date_features(df):
    df['Date'] = pd.to_datetime(df['Date']) 
    df['year'] = df['Date'].dt.year 
    df['month'] = df['Date'].dt.month 
    df['day'] = df['Date'].dt.day 
    del df['Date']
    return df
test=get_date_features(test)
train=get_date_features(train)

In [35]:
test.head(3)

,ID,Store,DayOfWeek,Holiday,year,month,day
0,1,12,6,0,2019,4,14
1,2,12,0,0,2019,4,15
2,3,12,1,0,2019,4,16


In [36]:
train.head(3)

,Store,DayOfWeek,Sales,Customers,Open,Promo,Holiday,year,month,day
0,474,5,4256,502,1,0,0.0,2019,4,13
1,862,5,2574,319,1,0,0.0,2019,4,13
2,108,5,4687,492,1,0,0.0,2019,4,13


In [37]:
train.shape

(963689, 10)

У нас лям строк, можем обучаться по кайфу.
Обучим катбуст , чтобы он мог предсказывать промо и праздники


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from catboost import Pool, CatBoostClassifier
from sklearn.metrics import f1_score
import numpy as np

In [39]:
train.dropna().reset_index(drop=True).describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,Holiday,year,month,day
count,832617.000000,832617.000000,832617.000000,832617.000000,832617.000000,832617.000000,832617.000000,832617.000000,832617.000000,832617.000000
mean,557.643010,3.004668,5757.529970,631.406915,0.824350,0.379715,0.037292,2017.542942,6.155985,15.488168
std,321.751505,1.994152,3893.132821,468.284912,0.380522,0.485316,0.189477,0.829093,3.711059,8.803100
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2016.000000,1.000000,1.000000
25%,279.000000,1.000000,3670.000000,398.000000,1.000000,0.000000,0.000000,2017.000000,3.000000,8.000000
50%,558.000000,3.000000,5736.000000,609.000000,1.000000,0.000000,0.000000,2017.000000,6.000000,15.000000
75%,837.000000,5.000000,7869.000000,839.000000,1.000000,1.000000,0.000000,2018.000000,10.000000,23.000000
max,1115.000000,6.000000,38722.000000,7388.000000,1.000000,1.000000,1.000000,2019.000000,12.000000,31.000000


In [40]:
del test['ID']

In [41]:
def predict_parametr(train_df, test_df, param):
    train_df=train_df.dropna().reset_index(drop=True)
    cat_cols=['Store', 'DayOfWeek', 'Holiday', 'year', 'month', 'day']
    
    for col in cat_cols:
        train_df[col]=train_df[col].astype('str')
        test_df[col]=test_df[col].astype('str')
    cols = test_df.columns
    X_train, X_test, y_train, y_test = train_test_split(train_df[cols], train_df[param], test_size=0.3, random_state=1)
    
    
    train_dataset_class = Pool(data=X_train,
                     label=y_train,
                     cat_features=cat_cols  #cat_features=[i for i in range(190,230)]  
                    )
    
    model = CatBoostClassifier(iterations=100)

    model.fit(train_dataset_class)
    
    preds_class = model.predict(X_test)
    
    print('F мера: ' ,f1_score(y_test, preds_class, average='macro'))
    
    answer= model.predict(test_df)
    return answer


open_status= predict_parametr(train_df=train, test_df=test, param='Open')


Learning rate set to 0.5
0:	learn: 0.3227781	total: 260ms	remaining: 25.8s
1:	learn: 0.1846969	total: 377ms	remaining: 18.4s
2:	learn: 0.1157546	total: 547ms	remaining: 17.7s
3:	learn: 0.0785900	total: 670ms	remaining: 16.1s
4:	learn: 0.0579977	total: 784ms	remaining: 14.9s
5:	learn: 0.0371688	total: 970ms	remaining: 15.2s
6:	learn: 0.0335759	total: 1.31s	remaining: 17.4s
7:	learn: 0.0396795	total: 1.42s	remaining: 16.3s
8:	learn: 0.0265432	total: 1.57s	remaining: 15.9s
9:	learn: 0.0238011	total: 1.67s	remaining: 15.1s
10:	learn: 0.0202095	total: 1.77s	remaining: 14.3s
11:	learn: 0.0194503	total: 1.91s	remaining: 14s
12:	learn: 0.0185170	total: 2s	remaining: 13.4s
13:	learn: 0.0179836	total: 2.09s	remaining: 12.8s
14:	learn: 0.0177834	total: 2.2s	remaining: 12.5s
15:	learn: 0.0174961	total: 2.29s	remaining: 12.1s
16:	learn: 0.0171848	total: 2.4s	remaining: 11.7s
17:	learn: 0.0169649	total: 2.53s	remaining: 11.5s
18:	learn: 0.0168682	total: 2.63s	remaining: 11.2s
19:	learn: 0.0167718	to

In [42]:
## Мы получили ох
print(len(open_status))
test['Open'] = open_status
test.columns

13104


Index(['Store', 'DayOfWeek', 'Holiday', 'year', 'month', 'day', 'Open'], dtype='object')

In [46]:
def predict(train_df):
    train_df=train_df.dropna().reset_index(drop=True)
    cols=['Store', 'DayOfWeek', 'Holiday', 'year', 'month', 'day']
    
    X_train, X_test, y_train, y_test = train_test_split(train_df[cols], train_df['Sales'], test_size=0.3, random_state=1)

    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 5)]
    # Number of features to consider at every split
    max_features = ['auto', 'sqrt']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(2, 30, num=3)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}
    
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 4, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    rf_random.fit(X_train, y_train)
    
    print(rf_random.best_params_)
    
    y_pred=rf_random.predict(X_test)
    
    print(mean_squared_error(y_test, y_pred) )
    
    
    return rf_random

predict_model = predict(train)



Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 19.2min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 19.2min finished


{'n_estimators': 1550, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 16, 'bootstrap': True}
6253033.092735921


In [48]:
print({'n_estimators': 650, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True})

print()
print('Это бест параметры для леса')
print()
print('Вторые топ параметры:')
print({'n_estimators': 1550, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 16, 'bootstrap': True})

{'n_estimators': 650, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 30, 'bootstrap': True}

Это бест параметры для леса

Вторые топ параметры:
{'n_estimators': 1550, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 16, 'bootstrap': True}


In [49]:
regr = RandomForestRegressor(max_depth=16, random_state=1,n_estimators=1550,
                            min_samples_split=2, min_samples_leaf=1,
                            max_features='sqrt',bootstrap=True)



first_train = train.dropna()
regr.fit(first_train[['Store', 'DayOfWeek', 'Holiday', 'year', 'month', 'day', 'Open']], first_train['Sales'])





RandomForestRegressor(max_depth=16, max_features='sqrt', n_estimators=1550,
                      random_state=1)

In [50]:
prediction=regr.predict(test[['Store', 'DayOfWeek', 'Holiday', 'year', 'month', 'day', 'Open']])
test1 = pd.read_csv(dir_+"test.csv")

test['ID']=test1['ID']
submission = test['ID'].to_frame()
submission['prediction'] = prediction
submission.to_csv(dir_+'sampleSubmission2.csv', index=None)

на сабмите получил 2766.90952 (первые параметры леса)

На сабмите получил 2622.79483 (вторые параметры леса ) побил бейзлайн. На 6ом месте лидерборда


#### Конец